In [2]:
#Importando librerias
import json
import numpy as np
import pandas as pd
import ast
import gzip
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
# Definimos una función leer_Json_gzip() que lee los archivos 'comprimidos.json.gz` y devuelve una lista de diccionarios Python.
def leer_json_gzip(nombreArchivo):

    # Abrimos el archivo users_items.json.gz en modo de lectura y decodificamos el contenido del archivo.
    with gzip.open(nombreArchivo, 'rt', encoding='utf-8') as miArchivo:

        # Recorremos el contenido del archivo y evaluamos cada línea como una expresión Python.
        # El resultado de la evaluación es un diccionario Python.
        # La función strip() elimina los espacios en blanco al principio y al final de la línea.
        return [ast.literal_eval(line.strip()) for line in miArchivo]
    #Usamos ast.literal_eval para convertir cada linea en un diccionario y lo agregamos a la lista

In [4]:
dfreviews1 = leer_json_gzip('C:/Users/Edgar/OneDrive/Escritorio/ProyectoFinal1HenryNuevo/user_reviews.json.gz')

dfreviews2= pd.DataFrame(dfreviews1)
dfreviews2.head(2)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."


In [5]:
# Separar las variables que vienen en el diccionario de reviews.
f=[]
for i in dfreviews2.values:
    f.append(i[2])

dfreviews3 = pd.DataFrame([review for sublist in f for review in sublist])

In [6]:
dfreviews3.shape #registros de la tabla review desanidada
dfreviews2.shape #registros de la tabla review sin desanidadar

(25799, 3)

In [7]:
dfreviews4 = dfreviews2.explode('reviews')
dfreviews4.shape

(59333, 3)

In [8]:
dfreviews4.head(3)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."


In [9]:
#reseteo el index
dfreviews4.reset_index(inplace=True)
dfreviews3.reset_index(inplace=True)

In [10]:
#unifico con user id repetidos
dfreviews5 = pd.concat([dfreviews4,dfreviews3], axis=1)
dfreviews5 = dfreviews5.drop(columns = ['reviews','index','funny','user_url','last_edited'])
dfreviews5.head()

,user_id,posted,item_id,helpful,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [11]:
## REVISO DUPLICADOS
duplicados=dfreviews5.loc[dfreviews5.duplicated()]
duplicados=duplicados.sort_values('user_id')
duplicados.head(5)

,user_id,posted,item_id,helpful,recommend,review
25139,0-3-0,"Posted May 2, 2014.",244850,No ratings yet,True,Like playing minecraft in space very addictive...
8027,111222333444555666888,Posted February 1.,213670,No ratings yet,True,"Jogo muito bom e engraçado,especialmente para ..."
8028,111222333444555666888,Posted February 1.,730,No ratings yet,True,"Jogo muito viciante e variado...Enfim,recomend..."
8029,111222333444555666888,"Posted November 21, 2015.",307880,No ratings yet,True,Good game when you can make your house.Hahaha!
8030,111222333444555666888,"Posted November 6, 2015.",8870,No ratings yet,True,Esse jogo me impressionou.Comprei o triple pac...


In [12]:
nulos= dfreviews5.isnull().sum()
nulos # son productos de la union entre ambas tablas

user_id       0
posted       28
item_id      28
helpful      28
recommend    28
review       28
dtype: int64

In [13]:
dfreviews5 = dfreviews5.dropna().reset_index(drop=True)  # QUITAR LOS NULOS. 

In [14]:
dfreviews5['posted'] = dfreviews5['posted'].str.extract(r'Posted([\w\s\d,]+)') #Reemplazo la palabra Posted por espacio vacio
dfreviews5['year'] = dfreviews5['posted'].str.extract(r'(\d{4})').fillna('0') #extraigo el año y relleno los nan por 0
dfreviews5=dfreviews5.drop(columns=['posted']) #elimino la columna

In [15]:
dfreviews5.head(3)
dfreviews5.dtypes

user_id      object
item_id      object
helpful      object
recommend    object
review       object
year         object
dtype: object

In [16]:
dfreviews5.apply(lambda x: pd.api.types.infer_dtype(x.values))

user_id       string
item_id       string
helpful       string
recommend    boolean
review        string
year          string
dtype: object

### Feature Engineering: En el dataset user_reviews se incluyen reseñas de juegos hechos por distintos usuarios. Debes crear la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos de machine learning y el análisis de datos. De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1.

In [17]:
#si la polaridad va de 1 a -1, siendo 1 positivo, 0 neutro y -1 negativo, hay que realizar una partición eficiente
#para que no haya un desbalance en la distribución de la nueva variable, por lo cual siendo 0.33, la distancia entre
#cada nodo es entre 0.66/0.67

#luego de ciertas validaciones, comentarions como  best zombie game i have played, daba un 1, asi que no seria neutral
def analizar_sentimiento(variable):
    if variable is None:
        return 1 #Neutral
    analisis = TextBlob(variable)
    polaridad = analisis.sentiment.polarity
    if polaridad >= 0.25:
        return 2 #Positivo
    elif polaridad <= -0.33:
        return 0 #Negativo
    else:
        return 1 #Neutral

In [18]:
dfreviews5["sentiment_analisis"]= dfreviews5["review"].astype(str).apply(analizar_sentimiento)
dfreviews5.head(3)

,user_id,item_id,helpful,recommend,review,year,sentiment_analisis
0,76561197970982479,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011,1
1,76561197970982479,22200,No ratings yet,True,It's unique and worth a playthrough.,2011,2
2,76561197970982479,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2011,1


In [19]:
def sentiment_calc(text):
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return None

dfreviews5['polaridad'] = dfreviews5['review'].apply(sentiment_calc)

In [20]:
dfreviews5["sentiment_analisis"]= dfreviews5["review"].astype(str).apply(analizar_sentimiento)
dfreviews5.head(3)

,user_id,item_id,helpful,recommend,review,year,sentiment_analisis,polaridad
0,76561197970982479,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011,1,0.174444
1,76561197970982479,22200,No ratings yet,True,It's unique and worth a playthrough.,2011,2,0.337500
2,76561197970982479,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2011,1,0.050000


In [21]:
def sentiment_calc(text):
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return None

dfreviews5['polaridad'] = dfreviews5['review'].apply(sentiment_calc)

In [22]:
dfreviews5.head(5)
#adjunto la polaridad para ver que tan eficientes son los numeros

,user_id,item_id,helpful,recommend,review,year,sentiment_analisis,polaridad
0,76561197970982479,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011,1,0.174444
1,76561197970982479,22200,No ratings yet,True,It's unique and worth a playthrough.,2011,2,0.337500
2,76561197970982479,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2011,1,0.050000
3,js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014,1,0.114583
4,js41637,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2013,1,-0.036111


In [23]:
# hacemos algunas pruebas al azar
dfreviews5["review"][25]

'If you like defending dungeons then I suppose this game would suit your needs.'

In [24]:
dfreviews5["sentiment_analisis"][25]

0

In [25]:
dfreviews5["review"][1183]


'best zombie game i have played'

In [26]:
dfreviews5["sentiment_analisis"][1183]

2

In [27]:
# ya que vimos que esta correcto. quitamos la columna review para que no tengamos tantos datos y se pueda cargar facilmente
dfreviews5 = dfreviews5.drop(columns=["review"])
dfreviews5.head(2)

,user_id,item_id,helpful,recommend,year,sentiment_analisis,polaridad
0,76561197970982479,1250,No ratings yet,True,2011,1,0.174444
1,76561197970982479,22200,No ratings yet,True,2011,2,0.337500


In [28]:
new_parquetreviews_file_path = 'C:/Users/Edgar/OneDrive/Escritorio/ProyectoFinal1HenryNuevo/ArchivosFinales/dfreviews.parquet'

# Guardar el DataFrame en formato Parquet
dfreviews5.to_parquet(new_parquetreviews_file_path, index=False, compression="gzip")

In [29]:
# Tomar una muestra aleatoria del 10% del dataset para la API
dfreviewsAPI = dfreviews5.sample(frac=0.1, random_state=42)

# Guardar la muestra en la ruta especificada
ruta_salida = 'C:/Users/Edgar/OneDrive/Escritorio/ProyectoFinal1HenryNuevo/ArchivosAPI/dfreviewsAPI.parquet'
dfreviewsAPI.to_parquet(ruta_salida, engine='pyarrow')

print(f'Se ha guardado una muestra del 10% del dataset en {ruta_salida}')

Se ha guardado una muestra del 10% del dataset en C:/Users/Edgar/OneDrive/Escritorio/ProyectoFinal1HenryNuevo/ArchivosAPI/dfreviewsAPI.parquet
